# B"H

## Categorical Cross-Entropy Loss Derivative


We will not go thru the mathematical derivation of the Categorical Cross-Entropy loss here. 

Perhaps later this will be done here thoroughly. See the book, pages 215-219, for details. 

<br>

---

Over here, we'll cut to the chase.

The derivative of this loss function **with respect to its inputs** equals the negative ground-truth vector, divided by the vector of the predicted values (which is also the output vector of the softmax function):

![](https://drive.google.com/uc?id=14mHq6DyVP8p44UITAq7UyVDxASc6Yhwq)

## Softmax Activation Derivative

As above, we will not go thru the mathematical derivation of the Softmax Activation here. 

Perhaps later this will be done here thoroughly. See the book, pages 220-229, for details. 

Note, the calculation is a bit complicated.